Konversi LEMI ke IAGA2002

In [1]:
from datetime import timedelta,datetime
import pandas as pd
from scipy import signal
import math
import numpy as np

Koefisien Gauss filter untuk data 1 detik

In [2]:
gauss_1s = [0.00045933, 0.00054772, 0.00065055, 0.00076964, 0.00090693,
        0.00106449, 0.00124449, 0.00144918, 0.00168089, 0.00194194, 0.00223468,
        0.00256140, 0.00292430, 0.00332543, 0.00376666, 0.00424959, 0.00477552,
        0.00534535, 0.00595955, 0.00661811, 0.00732042, 0.00806530, 0.00885090,
        0.00967467, 0.01053338, 0.01142303, 0.01233892, 0.01327563, 0.01422707,
        0.01518651, 0.01614667, 0.01709976, 0.01803763, 0.01895183, 0.01983377,
        0.02067480, 0.02146643, 0.02220039, 0.02286881, 0.02346437, 0.02398040,
        0.02441104, 0.02475132, 0.02499727, 0.02514602, 0.02519580, 0.02514602,
        0.02499727, 0.02475132, 0.02441104, 0.02398040, 0.02346437, 0.02286881,
        0.02220039, 0.02146643, 0.02067480, 0.01983377, 0.01895183, 0.01803763,
        0.01709976, 0.01614667, 0.01518651, 0.01422707, 0.01327563, 0.01233892,
        0.01142303, 0.01053338, 0.00967467, 0.00885090, 0.00806530, 0.00732042,
        0.00661811, 0.00595955, 0.00534535, 0.00477552, 0.00424959, 0.00376666,
        0.00332543, 0.00292430, 0.00256140, 0.00223468, 0.00194194, 0.00168089,
        0.00144918, 0.00124449, 0.00106449, 0.00090693, 0.00076964, 0.00065055,
        0.00054772, 0.00045933]

Koefisien Gauss untuk data 10 detik

In [3]:
gauss_10s = [0.00178860, 0.01060471, 0.04233562, 0.11379931, 0.20596804,
             0.25100743, 0.20596804, 0.11379931, 0.04233562, 0.01060471,
             0.00178860]

Fungsi untuk edit header IAGA-2002

In [4]:
def split(word):
	return [char for char in word]

def header(variable):
    staname_init = list(np.tile(' ',45))
    data_out = split(variable)
    for i in range(len(data_out)): staname_init[i]=data_out[i]
    station = ''.join(staname_init)
    return station

Parameter stasiun

In [5]:
# Parameter stasiun
nama_station = 'Tuntungan'
kode_IAGA = 'TUN'
latitude = '03 30 01.4 N'
longitude = '98 33 51.6 E'
elevation = '86'

Input data

In [6]:
folder = 'data'
file1 = 'tuntungan_lemi_raw_20210430.txt'
file2 = 'tuntungan_lemi_raw_20210501.txt'
proton1 = 'tuntungan_proton_raw_20210430.120'
proton2 = 'tuntungan_proton_raw_20210501.121'

Fungsi untuk membaca data LEMI

In [7]:
def read_lemi(filename):
    # Read data lemi
    df = pd.read_csv(filename, sep='\s+', names=['year', 'month', 'day', 'hour', 'minute', 'second', 'Bx', 'By', 'Bz', 'Te', 'Tf', 'A', 'N', 'FL', 'int'])
    # Set date time column
    df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute', 'second']])
    # Select important data only
    df = df[['date', 'Bx', 'By', 'Bz', 'Te', 'Tf']]
    # Set date time as index
    df = df.set_index('date')
    
    return df

Fungsi untuk membaca data proton

In [8]:
def read_proton(filename):
    # Read data proton
    df = pd.read_csv(filename, sep="\s+|:", engine='python', decimal=',', names=['proton', 'id', 'id2', 'date', 'hour', 'minute', 'second'])
    # Set date time column
    df['date'] = pd.to_datetime(df['date'], format="%m.%d.%y")
    df['date'] = df['date'] + pd.to_timedelta(df['hour'], unit='h')
    df['date'] = df['date'] + pd.to_timedelta(df['minute'], unit='m')
    df['date'] = df['date'] + pd.to_timedelta(df['second'], unit='s')
    # Bagi proton dengan 1000
    df['Bf'] = df['proton']/1000
    # Select important data only
    df = df[['date', 'Bf']]
    # Set date time as index
    df = df.set_index('date')
    # Remove zero value
    df = df[df['Bf'] != 0]
    
    return df

Fungsi untuk konversi data LEMI

In [9]:
def gauss_filter_1s(df, comp):
    if comp == 'X':
        data_comp = df['Bx']
    elif comp =='Y':
        data_comp = df['By']
    elif comp == 'Z':
        data_comp = df['Bz']
        
    # Convert to 1 minute value using gaussian filter
    out = [0]*1440
    for i in range(0, 1440):
        num_of_points = 0
        result = 0
        filter_sum = 0
        data = data_comp.iloc[86355+(60*i):86446+(60*i)]
        for j in range(0, 91):
            if math.isnan(data.iloc[j]) == False:
                result += data[j]*gauss_1s[j]
                filter_sum += gauss_1s[j]
                num_of_points += 1
    
        if num_of_points >= 0.9*91:
            out[i] = result/filter_sum
        else:
            out[i] = np.nan
            
    return out

Fungsi untuk konversi data proton

In [10]:
def gauss_filter_10s(df):
    data_comp = df['Bf']
    # Convert to 1 minute value using gaussian filter
    out = [0]*1440
    for i in range(0, 1440):
        num_of_points = 0
        result = 0
        filter_sum = 0
        data = data_comp.iloc[8635+(6*i):8646+(6*i)]
        for j in range(0, 11):
            if math.isnan(data.iloc[j]) == False:
                result += data[j]*gauss_10s[j]
                filter_sum += gauss_10s[j]
                num_of_points += 1    

        if num_of_points >= 0.9*11:
            out[i] = result/filter_sum
        else:
            out[i] = np.nan
            
    return out

Baca data

In [11]:
lemi1 = read_lemi(folder + '/'+ file1)
lemi2 = read_lemi(folder + '/'+ file2)
proton1 = read_proton(folder + '/'+ proton1)
proton2 = read_proton(folder + '/'+ proton2)

Tandai data kosong

In [12]:
lemi1 = lemi1.resample('1S').asfreq().reset_index()
lemi2 = lemi2.resample('1S').asfreq().reset_index()
proton1 = proton1.resample('10S').asfreq().reset_index()
proton2 = proton2.resample('10S').asfreq().reset_index()

Gabungkan data lemi dan proton

In [13]:
lemi = pd.concat([lemi1, lemi2]).set_index('date')
proton = pd.concat([proton1, proton2]).set_index('date')

Konversi ke 1 menit dengan gaussian filter

In [14]:
X_1m = np.array(gauss_filter_1s(lemi, 'X'))
Y_1m = np.array(gauss_filter_1s(lemi, 'Y'))
Z_1m = np.array(gauss_filter_1s(lemi, 'Z'))
F_1m = np.array(gauss_filter_10s(proton))
H_1m = np.sqrt(np.array(X_1m)**2+np.array(Y_1m)**2)
D_1m = 60*np.degrees(np.arcsin(Y_1m/H_1m)) 

Isi missing data dengan nilai 99999

In [15]:
X_1m[np.isnan(X_1m)] = 99999
Y_1m[np.isnan(Y_1m)] = 99999
Z_1m[np.isnan(Z_1m)] = 99999
F_1m[np.isnan(F_1m)] = 99999
H_1m[np.isnan(H_1m)] = 99999
D_1m[np.isnan(D_1m)] = 99999

Simpan ke IAGA2002

In [16]:
# Simpan ke IAGA2002
fileout = kode_IAGA+str(lemi.index[-1].year)+str(lemi.index[-1].month).zfill(2)+str(lemi.index[-1].day).zfill(2)+'vmin'+'.min'
f_iaga = open('out/' + fileout, 'w')

f_iaga.write(' Format                 IAGA-2002                                    |\n')
f_iaga.write(' Source of Data         BMKG                                         |\n')
f_iaga.write(' Station Name           %s|\n' %header(nama_station))
f_iaga.write(' IAGA Code              %s|\n' %header(kode_IAGA))
f_iaga.write(' Geodetic Latitude      %s|\n' %header(latitude))
f_iaga.write(' Geodetic Longitude     %s|\n' %header(longitude))
f_iaga.write(' Elevation              %s|\n' %header(elevation))
f_iaga.write(' Reported               DHZF                                         |\n')
f_iaga.write(' Sensor Orientation     XYZ                                          |\n')
f_iaga.write(' Digital Sampling       1 second                                     |\n')
f_iaga.write(' Data Interval Type     Filtered 1-minute (00:15-01:45)              |\n')
f_iaga.write(' Data Type              variation                                    |\n')
f_iaga.write('DATE       TIME         DOY     %sD      %sH      %sZ      %sF   |\n' %(kode_IAGA, kode_IAGA, kode_IAGA, kode_IAGA))

for i in range(0, 1440):
    body_iaga = '%s    %9.2f %9.2f %9.2f %9.2f\n' %((datetime(year=int(lemi.index[-1].year),month=int(lemi.index[-1].month),day=int(lemi.index[-1].day))+timedelta(minutes=i)).strftime("%Y-%m-%d %H:%M:00.000 %j"), D_1m[i], H_1m[i], Z_1m[i], F_1m[i])
    f_iaga.write(body_iaga)
f_iaga.close()